In [13]:
import pandas as pd
import os
staff = pd.read_csv('/Users/eloisefeng/Desktop/NU/Data Management/Staff.csv')
sales = pd.read_csv('/Users/eloisefeng/Desktop/NU/Data Management/Sales2.csv')
order_fact = pd.read_csv('/Users/eloisefeng/Desktop/NU/Data Management/Order_Fact.csv')
address = pd.read_csv('/Users/eloisefeng/Desktop/NU/Data Management/Employee_Addresses.csv')
payroll = pd.read_csv('/Users/eloisefeng/Desktop/NU/Data Management/Employee_Payroll2.csv')
donation = pd.read_csv('/Users/eloisefeng/Desktop/NU/Data Management/Employee_Donations.csv')

In [14]:
####NON-CORRELATED SUBQUERY

## Business Scenario #1
## Create a report that displays Job_Title for job groups with an average salary greater than the average salary of the company as a whole.

## First, Let's determine the overall company salary
staff.groupby('Job_Title').agg(MeanSalary=('Salary', 'mean'), Emps=('Employee_ID', 'count')).reset_index()
# If group by Job_Title, Pandas moves Job_Title out of the standard columns and into the Index, which is the bold labels on the left of the output
# that's why we need .reset_index() to convert it back into a column

,Job_Title,MeanSalary,Emps
0,Account Manager,46090.0,1
1,Accountant I,31563.0,5
2,Accountant II,37284.0,5
3,Accountant III,37492.5,2
4,Administration Manager,47415.0,2
...,...,...,...
127,Warehouse Assistant I,26261.5,10
128,Warehouse Assistant II,27295.0,4
129,Warehouse Assistant III,28197.5,2
130,Warehouse Assistant IV,29625.0,1


In [ ]:
## Now wrap the outer query around the subquery
avg_salary = staff['Salary'].mean()
staff_grouped = staff.groupby('Job_Title').agg(MeanSalary=('Salary', 'mean')).reset_index()
staff_grouped[staff_grouped['MeanSalary'] > avg_salary]
# Inner part: does not return the data, returns a list of "Yes/No" (True/False) answers for every single row
# Outer part: take that list of True/False answers, and give the actual rows from df_grouped where the answer was True

,Job_Title,MeanSalary
0,Account Manager,46090.000000
4,Administration Manager,47415.000000
5,Applications Developer I,42760.000000
6,Applications Developer II,47315.000000
7,Applications Developer IV,55751.666667
...,...,...
117,Technical Manager,48075.000000
124,Trainer III,39330.000000
125,Training Manager,48335.000000
126,Vice President,194885.000000


In [ ]:
## Busines Scenario #2
## Each month, the CEO sends a birthday card to each employee having a birthday in that month. 
## Create a report listing the names and addresses of employees with February birthdays.
# First Lets build out the subquery -- select the employee IDs from payroll table with February birth days
feb_emps = payroll[pd.to_datetime(payroll['Birth_Date']).dt.month == 2]
# convert the Birth_Date column into a real datetime object, not just text
feb_emps[['Employee_ID']]
# use couble [] to return a dataframe (the table)
# is only use single [], the result would be data only

,Employee_ID
7,120108
11,120112
13,120114
56,120157
58,120159
69,120170
72,120173
100,120261
102,120263
128,120664


In [ ]:
#Complete Code
## Create a report listing the names and addresses of employees with February birthdays.
feb_emps = payroll[pd.to_datetime(payroll['Birth_Date']).dt.month == 2]
result = pd.merge(address, feb_emps, on='Employee_ID', how='inner')
# pd.merge() is inner join by default
result[['Employee_ID', 'Employee_Name', 'City', 'Country']]

,Employee_ID,Employee_Name,City,Country
0,121038,"Anstey, David",Miami-Dade,US
1,120812,"Arruza, Fauver",Miami-Dade,US
2,120756,"Asta, Wendy",Philadelphia,US
3,121049,"Bataineh, Perrior",San Diego,US
4,120114,"Buddery, Jeannette",Sydney,AU
5,120713,"Campbell, Carston",San Diego,US
6,120263,"Cleverley, Bobby",San Diego,US
7,121113,"Costine, Karen",Miami-Dade,US
8,121132,"Digiorgio, Shia-Ling",San Diego,US
9,120112,"Glattback, Ellis",Melbourne,AU


In [ ]:
####CORRELATED SUBQUERY


## Business Scenario #3
## Create a report showing Employee_ID and Job_Title columns of all sales personnel who did not make any sales.
## The table Sales contains Employee_ID and Job_Title columns for all sales personnel.
## The table Order_Fact holds information about all sales, and the Employee_ID column contains the employee identifier of the staff member who made the sale.
sold_id = order_fact['Employee_ID'].unique()
no_sale = sales[~sales['Employee_ID'].isin(sold_id)]
# ~ means NOT
no_sale[['Employee_ID', 'Job_Title']]

,Employee_ID,Job_Title
0,120102,Sales Manager
1,120103,Sales Manager
6,120125,Sales Rep. IV
7,120126,Sales Rep. II
10,120129,Sales Rep. III
...,...,...
159,121138,Sales Rep. I
160,121139,Sales Rep. II
162,121143,Senior Sales Manager
163,121144,Sales Manager


In [ ]:
# or use merge
merged = pd.merge(sales, order_fact, on='Employee_ID', how='left', indicator=True)
# indicator=True: add a special tracking column (usually named _merge) to the result
# This column tells you where each row came from
no_sale = merged[merged['_merge'] == 'left_only']
# == 'left_only': filter for rows that exist in the left table
no_sale[['Employee_ID', 'Job_Title']]

,Employee_ID,Job_Title
0,120102,Sales Manager
1,120103,Sales Manager
21,120125,Sales Rep. IV
22,120126,Sales Rep. II
40,120129,Sales Rep. III
...,...,...
375,121138,Sales Rep. I
376,121139,Sales Rep. II
378,121143,Senior Sales Manager
379,121144,Sales Manager


In [ ]:
#### SUBQUERY EXERCISE


##Create a report listing the employee identifier and the first name followed by the last name for all managers in the US.
# with regex
merge_file = pd.merge(address, staff, on='Employee_ID', how='left')
us_manager = merge_file[(merge_file['Country'] == 'US') & (merge_file['Job_Title'].str.endswith('Manager'))].copy()
# us_manager = merge_file[...]: Python doesn't create new data, just creates a window looking at specific rows of merge_file
# .copy(): forces Python to paste the data into a new memory slot, and us_manager is its own independent object
us_manager['Name'] = us_manager['Employee_Name'].str.replace(
    r'^(.*),\s*(.*)$', r'\2 \1', regex=True
)
# regex=True means we are using regex instead of looking for the exact string '^(.*),\s*(.*)$'
us_manager[['Employee_ID', 'Name']]

,Employee_ID,Name
8,120798,Elizabeth Ardskin
31,120735,Brenda Bilobran
45,120724,Hampie Brown
52,121144,Renee Capachietti
80,120679,Chrisy Cutucache
97,120714,Robert Dinley
99,120668,Thyland Dolan
101,120785,Damesha Donnell
119,121143,Louis Favaron
174,121001,Tony House


In [47]:
# or with split()
merge_file = pd.merge(address, staff, on='Employee_ID', how='left')
split_name = merge_file['Employee_Name'].str.split(',', expand=True)
# without expand=True (default): returns a list inside a single column
# expand=True: converts the lists into a new dataframe with separate columns
merge_file['Name'] = split_name[1].str.strip() + ' ' + split_name[0].str.strip()
# .str.strip(): remove extra spaces
# careful with [] when selecting strings in split_name
merge_file[['Employee_ID', 'Name']]

,Employee_ID,Name
0,121044,Ray Abbott
1,120145,Sandy Aisbitt
2,120761,Tameaka Akinfolarin
3,120656,Salley Amos
4,121107,Rose Anger
...,...,...
419,120771,Wei Xing Moore
420,120102,Tom Zhou
421,120804,Ahmed Zied
422,120670,Odudu Zisek


In [35]:
## JOIN Exercise 1
## Display the name, city, and birth month of all Australian employees
address['Birth_Month'] = pd.to_datetime(payroll['Birth_Date'], format='%m/%d/%y').dt.month
au_emps = address[address['Country'] == 'AU']
au_emps[['Employee_Name', 'City', 'Birth_Month']]

,Employee_Name,City,Birth_Month
1,"Aisbitt, Sandy",Melbourne,8
16,"Bahlman, Sharon",Sydney,9
17,"Baker, Gabriele",Sydney,6
21,"Baran, Shanmuganathan",Sydney,7
22,"Barbis, Viney",Sydney,9
...,...,...,...
402,"Waal, Samantha",Sydney,9
413,"Wills, Matsuoka",Melbourne,2
414,"Wilson, Fang",Sydney,8
420,"Zhou, Tom",Melbourne,11


In [ ]:
## JOIN Exercise 2

## Each month, the CEO sends a birthday card to each employee having a birthday in that month.
## Create a report listing the names and addresses AND BIRTHDAY of employees with February birthdays.
feb_emps = payroll[pd.to_datetime(payroll['Birth_Date'], format='%m/%d/%y').dt.month == 2]
merge_birth = pd.merge(address, feb_emps, on='Employee_ID', how='right')
merge_birth['Address'] = (
    merge_birth['Street_Number'].astype(str) + ' ' +
    merge_birth['Street_Name'] + ', ' +
    merge_birth['City'] + ', ' +
    merge_birth['State'] + ' ' +
    merge_birth['Postal_Code'].astype(str) + ', ' +
    merge_birth['Country']
)
# use .astype(str) to turn integers into strings
# be careful with NA values as they will result the Address to be NA as a whole
# can use .str.cat() to ignore the NA values
merge_birth[['Employee_Name', 'Address', 'Birth_Date']]

,Employee_Name,Address,Birth_Date
0,"Gromek, Gladys","6 George Street, Melbourne, 3000, AU",2/23/84
1,"Glattback, Ellis","7 Vulture Street, Melbourne, 3174, AU",2/17/69
2,"Buddery, Jeannette","12 Hunua Road Papakura, Sydney, 1004, AU",2/8/44
3,"Karavdic, Leonid","121 Blackburn Road, Sydney, 2580, AU",2/21/86
4,"Phoumirath, Lynelle","166 Toorak Road, Sydney, 2119, AU",2/24/64
5,"Kingston, Alban","5 Buffalo Road, Sydney, 1042, AU",2/25/74
6,"Osborn, Hernani","8 Cannon Lane Whitby, Melbourne, 3350, AU",2/23/54
7,"Highpoint, Harry","18 Antler Ct, Miami-Dade, FL 33135, US",2/21/69
8,"Cleverley, Bobby","622 Boylan Ave, San Diego, CA 92054, US",2/10/64
9,"Senchak, Brock","1779 Greymist Ln, Philadelphia, PA 19142, US",2/18/54


In [58]:
## JOIN Exercise 3
## List the employee ID and gender for all married employees. Include the names of any charities to which the employee donates via the company program.
## Considerations:
## The table Employee_Payroll contains gender and marital status information.
## The table Employee_Donations contains records only for those employees who donate to a charity via the company program.
## Less than half of all employees are married
merge_donation = pd.merge(donation, payroll, on='Employee_ID', how='right')
married = merge_donation[merge_donation['Marital_Status'] == 'M']
married[['Employee_ID', 'Employee_Gender', 'Recipients']]

,Employee_ID,Employee_Gender,Recipients
2,120103,M,NaN
3,120104,F,NaN
5,120106,M,NaN
6,120107,F,NaN
8,120109,F,NaN
...,...,...,...
419,121144,F,NaN
420,121145,M,Save the Baby Animals
421,121146,F,NaN
422,121147,F,"Cuidadores Ltd. 50%, Mitleid International 50%"


In [61]:
###  ADDITIONAL PROBLEMS

# 1. Use the payroll and sales tables to create a list of the married sales employees
merge_married = pd.merge(payroll, sales, on='Employee_ID', how='right')
merge_married[merge_married['Marital_Status'] == 'M']

,Employee_ID,Employee_Gender,Salary_x,Birth_Date_x,Employee_Hire_Date,Employee_Term_Date,Marital_Status,Dependents,First_Name,Last_Name,Gender,Salary_y,Job_Title,Country,Birth_Date_y,Hire_Date
1,120103,M,87975,1/22/49,1/1/74,,M,1,Wilson,Dawes,M,87975,Sales Manager,AU,1/22/49,1/1/74
2,120121,F,26600,8/2/44,1/1/74,,M,1,Irenie,Elvish,F,26600,Sales Rep. II,AU,8/2/44,1/1/74
4,120123,F,26190,9/28/64,10/1/85,1/31/05,M,3,Kimiko,Hotstone,F,26190,Sales Rep. I,AU,9/28/64,10/1/85
5,120124,M,26480,5/13/59,3/1/79,,M,1,Lucian,Daymond,M,26480,Sales Rep. I,AU,5/13/59,3/1/79
6,120125,M,32040,12/6/54,3/1/79,7/31/04,M,2,Fong,Hofmeister,M,32040,Sales Rep. IV,AU,12/6/54,3/1/79
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,121138,M,27265,2/28/49,1/1/74,,M,1,Hershell,Tolley,M,27265,Sales Rep. I,US,2/28/49,1/1/74
160,121139,F,27700,8/19/59,7/1/87,,M,2,Diosdado,Mckee,F,27700,Sales Rep. II,US,8/19/59,7/1/87
162,121143,M,95090,11/26/69,7/1/97,,M,3,Louis,Favaron,M,95090,Senior Sales Manager,US,11/26/69,7/1/97
163,121144,F,83505,6/28/64,11/1/91,,M,3,Renee,Capachietti,F,83505,Sales Manager,US,6/28/64,11/1/91


In [62]:
# 2. Modify the query to limit to the Australian married managers
merge_married_au = pd.merge(payroll, sales, on='Employee_ID', how='right')
merge_married_au[(merge_married_au['Marital_Status'] == 'M') & (merge_married_au['Country'] == 'AU')]

,Employee_ID,Employee_Gender,Salary_x,Birth_Date_x,Employee_Hire_Date,Employee_Term_Date,Marital_Status,Dependents,First_Name,Last_Name,Gender,Salary_y,Job_Title,Country,Birth_Date_y,Hire_Date
1,120103,M,87975,1/22/49,1/1/74,,M,1,Wilson,Dawes,M,87975,Sales Manager,AU,1/22/49,1/1/74
2,120121,F,26600,8/2/44,1/1/74,,M,1,Irenie,Elvish,F,26600,Sales Rep. II,AU,8/2/44,1/1/74
4,120123,F,26190,9/28/64,10/1/85,1/31/05,M,3,Kimiko,Hotstone,F,26190,Sales Rep. I,AU,9/28/64,10/1/85
5,120124,M,26480,5/13/59,3/1/79,,M,1,Lucian,Daymond,M,26480,Sales Rep. I,AU,5/13/59,3/1/79
6,120125,M,32040,12/6/54,3/1/79,7/31/04,M,2,Fong,Hofmeister,M,32040,Sales Rep. IV,AU,12/6/54,3/1/79
8,120127,F,28100,1/4/79,11/1/98,,M,2,Sharryn,Clarkson,F,28100,Sales Rep. II,AU,1/4/79,11/1/98
11,120130,M,26955,12/14/84,5/1/06,,M,2,Kevin,Lyon,M,26955,Sales Rep. I,AU,12/14/84,5/1/06
15,120134,M,28015,6/6/49,1/1/74,6/30/06,M,2,Sian,Shannan,M,28015,Sales Rep. II,AU,6/6/49,1/1/74
16,120135,M,32490,1/26/69,10/1/97,4/30/04,M,3,Alexei,Platts,M,32490,Sales Rep. IV,AU,1/26/69,10/1/97
17,120136,M,26605,9/16/79,2/1/03,,M,1,Atul,Leyden,M,26605,Sales Rep. I,AU,9/16/79,2/1/03


In [94]:
# 3. Use the staff and donation tables to create a report that shows the list of employee job titles 
# who made a donation by "Cash or Check" in the third quarter
import numpy as np
donation['Qtr3'] = donation['Qtr3'].replace(r'^\s*$', np.nan, regex=True)
# some of the NULL values are as " ". SQL can recognize it as NULL but not Python
donate = pd.merge(staff, donation, on='Employee_ID', how='inner')
cash = donate[(donate['Paid_By'] == 'Cash or Check') & (donate['Qtr3'].notna())]
# .notna() doesn't catch " " or 0
# ~.isnull() is the same with .notna()
unique_title = cash[['Job_Title']].drop_duplicates()
# .drop_duplicates() can get a table (dataframe)
# .unique() can only be used as cash[['Job_Title']].unique(), which will return a list (array)
unique_title

,Job_Title
3,Concession Director
8,Secretary II
9,Pricing Manager
15,Shipping Manager
17,Shipping Agent II
25,Purchasing Agent III
28,Purchasing Agent I
36,Accountant III
37,Accountant II
38,Financial Controller III
